In [3]:
!git init

Reinitialized existing Git repository in /home/matinm/chest X-ray/.git/


In [4]:
!git remote add origin https://github.com/MatinM-96/Chest-X-ray.git

error: remote origin already exists.


In [5]:
import numpy as np 

In [6]:
# In a Jupyter cell, the leading "!" runs a shell command
!pip -q install kaggle

# Upload your Kaggle API key (kaggle.json) to the notebook, or paste below:


In [7]:
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls -lah ~/.kaggle/kaggle.json


mv: cannot stat 'kaggle.json': No such file or directory
-rw------- 1 matinm users 64 Aug 18 09:12 /home/matinm/.kaggle/kaggle.json


In [8]:
!mkdir -p ~/.kaggle

In [9]:
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls -lah ~/.kaggle/kaggle.json# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = ""

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "nih-chest-xrays/data",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

mv: cannot stat 'kaggle.json': No such file or directory
ls: cannot access '/home/matinm/.kaggle/kaggle.json#': No such file or directory
ls: cannot access 'Install': No such file or directory
ls: cannot access 'dependencies': No such file or directory
ls: cannot access 'as': No such file or directory
ls: cannot access 'needed:': No such file or directory


ModuleNotFoundError: No module named 'kagglehub'

In [11]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = ""

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "nih-chest-xrays/data",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

ModuleNotFoundError: No module named 'kagglehub'

In [ ]:
!pip install kagglehub

In [ ]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = ""

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "nih-chest-xrays/data",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

In [ ]:
import os, json
with open("./kaggle.json", "r") as f:   # change to "./kaggle.json" if it's beside the notebook
    creds = json.load(f)
os.environ["KAGGLE_USERNAME"] = creds["username"]
os.environ["KAGGLE_KEY"] = creds["key"]


In [33]:
# pip install kagglehub[pandas-datasets] pandas

import pandas as pd
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Pick a file from the dataset to read directly into pandas:
file_path = "Data_Entry_2017.csv"   # NIH Chest X-rays: labels/metadata CSV

df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "nih-chest-xrays/data",
    file_path,
    pandas_kwargs={"low_memory": False}  # optional pandas args
)




import numpy as np

# NIH canonical 14 labels
LABELS = [
    'Atelectasis','Cardiomegaly','Effusion','Infiltration','Mass','Nodule',
    'Pneumonia','Pneumothorax','Consolidation','Edema','Emphysema',
    'Fibrosis','Pleural_Thickening','Hernia'
]


# Convert “Finding Labels” to multi-hot vectors
def to_multi_hot(lbl_str):
    y = np.zeros(len(LABELS), dtype=np.float32)
    if isinstance(lbl_str, str) and lbl_str != "No Finding":
        for t in lbl_str.split("|"):
            if t in LABELS:
                y[LABELS.index(t)] = 1.0
    return y





# 1. Make an empty list
all_labels = []

# 2. Loop over each row in the DataFrame
for label_str in df["Finding Labels"].astype(str):
    # Convert string like "Cardiomegaly|Effusion" to a multi-hot vector
    y = to_multi_hot(label_str)
    # Add it to the list
    all_labels.append(y)

# 3. Convert the list of vectors into one big numpy array
Y = np.stack(all_labels, axis=0)

print("Shape of Y:", Y.shape)


# Quick label stats
pos_per_class = Y.sum(axis=0)
print("Positives per class:", dict(zip(LABELS, pos_per_class.astype(int))))

# Patient-level split to avoid leakage
df["Patient ID"] = df["Patient ID"].astype(str)
bucket = df["Patient ID"].apply(lambda x: hash(x) % 10)  # 0..9
train_df = df[bucket < 8].reset_index(drop=True)
val_df   = df[bucket == 8].reset_index(drop=True)
test_df  = df[bucket == 9].reset_index(drop=True)

len(train_df), len(val_df), len(test_df)


#how to downalod the data 

# import kagglehub

# path = kagglehub.dataset_download("nih-chest-xrays/data")
# print("Dataset downloaded to:", path)

# IMAGES_ROOT = f"{path}/images"


# !pip install kagglehub







import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# TODO: set this to where images are accessible in your environment.
# For Kaggle Notebooks, inspect the dataset sidebar to confirm the path.



imagePath =  "./chest-xray/images"

IMG_SIZE = 384
train_tfms = transforms.Compose([
    transforms.Resize(int(IMG_SIZE*1.1)),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
val_tfms = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

def row_to_multi_hot(row):
    return torch.tensor(to_multi_hot(row["Finding Labels"]), dtype=torch.float32)

class ChestXray(Dataset):
    def __init__(self, df, root, tfm):
        self.df = df.reset_index(drop=True)
        self.root = root
        self.tfm = tfm
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        r = self.df.iloc[i]
        img_path = os.path.join(self.root, r["Image Index"])
        img = Image.open(img_path).convert("RGB")
        x = self.tfm(img)
        y = row_to_multi_hot(r)
        return x, y

train_ds = ChestXray(train_df, IMAGES_ROOT, train_tfms)
val_ds   = ChestXray(val_df, IMAGES_ROOT, val_tfms)

train_dl = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
val_dl   = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

len(train_ds), len(val_ds)




import torch, torch.nn as nn
from torchvision import models
from sklearn.metrics import roc_auc_score
from rich.progress import track

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Model
model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
in_features = model.classifier.in_features
model.classifier = nn.Linear(in_features, len(LABELS))
model = model.to(DEVICE)

# Loss with optional class weights (handles imbalance)
# pos_weight = (N - pos) / pos  per class
N = len(train_df)
pos = torch.tensor([(train_df["Finding Labels"].astype(str).str.contains(l)).sum() for l in LABELS], dtype=torch.float32)
pos = torch.clamp(pos, min=1.0)
pos_weight = (N - pos) / pos
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight.to(DEVICE))

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))

def evaluate(model, loader):
    model.eval()
    ys, ps = [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            p = torch.sigmoid(model(xb))
            ys.append(yb.cpu()); ps.append(p.cpu())
    ys = torch.cat(ys, 0).numpy()
    ps = torch.cat(ps, 0).numpy()
    per_cls = []
    for c in range(len(LABELS)):
        try:
            per_cls.append(roc_auc_score(ys[:,c], ps[:,c]))
        except ValueError:
            per_cls.append(float("nan"))
    return float(np.nanmean(per_cls)), dict(zip(LABELS, per_cls))

best_auc, best_path = -1.0, "densenet121_best.pt"
EPOCHS = 10

for ep in range(1, EPOCHS+1):
    model.train()
    for xb, yb in track(train_dl, description=f"Epoch {ep}/{EPOCHS}"):
        xb, yb = xb.to(DEVICE, non_blocking=True), yb.to(DEVICE, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):
            logits = model(xb)
            loss = criterion(logits, yb)
        scaler.scale(loss).backward()
        scaler.step(optimizer); scaler.update()
    scheduler.step()

    mean_auc, per_cls = evaluate(model, val_dl)
    print(f"Val mean AUROC: {mean_auc:.4f}")
    if mean_auc > best_auc:
        best_auc = mean_auc
        torch.save(model.state_dict(), best_path)
        print(f"Saved new best to {best_path}")





/tmp/ipykernel_152/2672883383.py:10: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


Shape of Y: (112120, 14)
Positives per class: {'Atelectasis': 11559, 'Cardiomegaly': 2776, 'Effusion': 13317, 'Infiltration': 19894, 'Mass': 5782, 'Nodule': 6331, 'Pneumonia': 1431, 'Pneumothorax': 5302, 'Consolidation': 4667, 'Edema': 2303, 'Emphysema': 2516, 'Fibrosis': 1686, 'Pleural_Thickening': 3385, 'Hernia': 227}


/usr/local/lib/python3.10/dist-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/tmp/ipykernel_152/2672883383.py", line 131, in __getitem__
    img = Image.open(img_path).convert("RGB")
  File "/usr/local/lib/python3.10/dist-packages/PIL/Image.py", line 3247, in open
    fp = builtins.open(filename, "rb")
FileNotFoundError: [Errno 2] No such file or directory: 'chest-xray/Images/00020751_002.png'


In [31]:
# ---- 1) Point to your existing images ----
# Example paths — change to YOUR real folder:
# IMAGES_ROOT = "/data/nih/images"
# IMAGES_ROOT = "/srv/datasets/NIH-ChestXray/images"
  # <-- put your real path
imagePath = "./chest-xray"



# Quick sanity check (prints a few that must exist)
from pathlib import Path
missing = 0
for name in df["Image Index"].head(20):
    if not Path(IMAGES_ROOT, name).exists():
        print("Missing:", name)
        missing += 1
print("Missing among first 20:", missing)
assert Path(IMAGES_ROOT).exists(), f"Folder not found: {IMAGES_ROOT}"

# (Optional but helpful for any occasional corrupted file)
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


Missing: 00000001_000.png
Missing: 00000001_001.png
Missing: 00000001_002.png
Missing: 00000002_000.png
Missing: 00000003_000.png
Missing: 00000003_001.png
Missing: 00000003_002.png
Missing: 00000003_003.png
Missing: 00000003_004.png
Missing: 00000003_005.png
Missing: 00000003_006.png
Missing: 00000003_007.png
Missing: 00000004_000.png
Missing: 00000005_000.png
Missing: 00000005_001.png
Missing: 00000005_002.png
Missing: 00000005_003.png
Missing: 00000005_004.png
Missing: 00000005_005.png
Missing: 00000005_006.png
Missing among first 20: 20


AssertionError: Folder not found: ./chestX-ray

In [23]:
imagePath = "./chest X-ray/images"



In [24]:
imagePath

'./chest-xray/images'

In [28]:
import os
print(os.listdir("chestX-ray")[:10])


FileNotFoundError: [Errno 2] No such file or directory: 'chestX-ray'

In [29]:
IMAGES_ROOT = "./chestX-ray"
import os
print(os.listdir("chestX-ray")[:10])


FileNotFoundError: [Errno 2] No such file or directory: 'chestX-ray'

In [30]:
import os
print(os.listdir("chest-xray")[:10])


['images_007', 'ARXIV_V5_CHESTXRAY.pdf', 'images_002', 'images_006', 'test_list.txt', 'Data_Entry_2017.csv', 'images_003', 'images_009', 'BBox_List_2017.csv', 'images_012']


In [39]:
# ========================== SETUP ==========================
# pip installs (uncomment if needed)
# !pip install -q kagglehub pandas torch torchvision scikit-learn tqdm

import os
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.metrics import roc_auc_score, f1_score
from tqdm import tqdm

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

# ========================== 1) LOAD CSV ==========================
# If you already loaded df with kagglehub in earlier cells, keep it.
# Otherwise, assuming the CSV sits next to your image folders (per screenshot):
csv_path = Path("./chest-xray/Data_Entry_2017.csv")
assert csv_path.exists(), f"CSV not found at: {csv_path}"
df = pd.read_csv(csv_path)
print("df shape:", df.shape)

# ========================== 2) LABELS + MULTI-HOT ==========================
LABELS = [
    'Atelectasis','Cardiomegaly','Effusion','Infiltration','Mass','Nodule',
    'Pneumonia','Pneumothorax','Consolidation','Edema','Emphysema',
    'Fibrosis','Pleural_Thickening','Hernia'
]

def to_multi_hot(lbl_str):
    y = np.zeros(len(LABELS), dtype=np.float32)
    if isinstance(lbl_str, str) and lbl_str != "No Finding":
        for t in lbl_str.split("|"):
            if t in LABELS:
                y[LABELS.index(t)] = 1.0
    return y

# build Y (aligned to df rows)
Y = np.stack([to_multi_hot(s) for s in df["Finding Labels"].astype(str)], axis=0)
print("Y shape:", Y.shape)

# simple stats
pos_per_class = Y.sum(axis=0)
print("Positives per class:", dict(zip(LABELS, pos_per_class.astype(int))))

# ========================== 3) PATIENT-LEVEL SPLIT ==========================
df["Patient ID"] = df["Patient ID"].astype(str)
bucket = df["Patient ID"].apply(lambda x: hash(x) % 10)  # 0..9
train_df = df[bucket < 8].reset_index(drop=True)
val_df   = df[bucket == 8].reset_index(drop=True)
test_df  = df[bucket == 9].reset_index(drop=True)
print("split sizes:", len(train_df), len(val_df), len(test_df))

# ========================== 4) INDEX FILES ACROSS SUBFOLDERS ==========================
# Your screenshot: chestX-ray/chest-xray/images_001..images_012





from pathlib import Path

# Set this to the parent that directly contains images_001, images_002, ...
# Example 1 (from your screenshots):
# BASE = Path("./chestX-ray/chest-xray")
# Example 2 (from your absolute path snippet):
BASE = Path("./chest-xray")   # <-- change if needed

assert BASE.exists(), f"Base not found: {BASE}"

# Build: filename -> full path, handling the extra '/images/' level
name_to_path = {}
for p in BASE.glob("images_*/images/*.png"):
    name_to_path[p.name] = str(p)



print("Indexed files:", len(name_to_path))







# quick sanity: first 20 filenames should all be found
first20 = df["Image Index"].head(20).tolist()
missing20 = [n for n in first20 if n not in name_to_path]
print("Missing among first 20:", len(missing20))
if missing20:
    print("Example missing:", missing20[:5])

# ========================== 5) DATASET / LOADERS ==========================
IMG_SIZE = 384
train_tfms = transforms.Compose([
    transforms.Resize(int(IMG_SIZE*1.1)),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
val_tfms = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

def row_to_multi_hot(row):
    return torch.tensor(to_multi_hot(row["Finding Labels"]), dtype=torch.float32)

class ChestXray(Dataset):
    def __init__(self, df, index_map, tfm):
        self.df = df.reset_index(drop=True)
        self.idx = index_map
        self.tfm = tfm
    def __len__(self): 
        return len(self.df)
    def __getitem__(self, i):
        r = self.df.iloc[i]
        fname = r["Image Index"]
        img_path = self.idx.get(fname)
        if img_path is None:
            raise FileNotFoundError(f"Image not indexed: {fname}")
        img = Image.open(img_path).convert("RGB")
        x = self.tfm(img)
        y = row_to_multi_hot(r)
        return x, y

train_ds = ChestXray(train_df, name_to_path, train_tfms)
val_ds   = ChestXray(val_df,   name_to_path, val_tfms)
test_ds  = ChestXray(test_df,  name_to_path, val_tfms)

BATCH_SIZE = 32
NUM_WORKERS = 2  # set 0 if you see multiprocessing issues

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS, pin_memory=True)
val_dl   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
test_dl  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

print("Splits:", len(train_ds), len(val_ds), len(test_ds))

# ========================== 6) MODEL ==========================
model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
in_features = model.classifier.in_features
model.classifier = nn.Linear(in_features, len(LABELS))
model = model.to(DEVICE)

# ========================== 7) LOSS (pos_weight from TRAIN) ==========================
train_multi = np.vstack(train_df["Finding Labels"].astype(str).map(
    lambda s: np.array(to_multi_hot(s), dtype=np.float32)
).values)
pos = train_multi.sum(axis=0)  # per-class positives in TRAIN
N = len(train_df)
pos = np.clip(pos, 1.0, None)
pos_weight = torch.tensor((N - pos) / pos, dtype=torch.float32, device=DEVICE)

criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))

# ========================== 8) EVAL FUNCTION ==========================
def evaluate(model, loader, threshold=0.5):
    model.eval()
    ys, ps = [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            p = torch.sigmoid(model(xb))
            ys.append(yb.cpu()); ps.append(p.cpu())
    ys = torch.cat(ys, 0).numpy()
    ps = torch.cat(ps, 0).numpy()

    # AUROC per class
    aurocs = []
    for c in range(len(LABELS)):
        y_c, p_c = ys[:, c], ps[:, c]
        try:
            aurocs.append(roc_auc_score(y_c, p_c))
        except ValueError:
            aurocs.append(np.nan)
    mean_auc = float(np.nanmean(aurocs))

    # F1s at threshold
    preds = (ps >= threshold).astype("int32")
    micro_f1 = f1_score(ys, preds, average="micro", zero_division=0)
    macro_f1 = f1_score(ys, preds, average="macro", zero_division=0)
    return mean_auc, dict(zip(LABELS, aurocs)), micro_f1, macro_f1

# ========================== 9) TRAIN ==========================
EPOCHS = 10
best_auc, best_path = -1.0, "densenet121_best.pt"

for ep in range(1, EPOCHS+1):
    model.train()
    running = 0.0
    for xb, yb in tqdm(train_dl, desc=f"Epoch {ep}/{EPOCHS}"):
        xb, yb = xb.to(DEVICE, non_blocking=True), yb.to(DEVICE, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):
            logits = model(xb)
            loss = criterion(logits, yb)
        scaler.scale(loss).backward()
        scaler.step(optimizer); scaler.update()
        running += loss.item() * xb.size(0)
    scheduler.step()
    train_loss = running / len(train_ds)

    val_mean_auc, _, val_micro_f1, val_macro_f1 = evaluate(model, val_dl)
    print(f"[Val] mean AUROC={val_mean_auc:.4f} | microF1={val_micro_f1:.4f} | macroF1={val_macro_f1:.4f} | train_loss={train_loss:.4f}")
    if val_mean_auc > best_auc:
        best_auc = val_mean_auc
        torch.save(model.state_dict(), best_path)
        print(f"✅ Saved new best to {best_path}")

# ========================== 10) TEST ==========================
if Path(best_path).exists():
    model.load_state_dict(torch.load(best_path, map_location=DEVICE))

test_mean_auc, test_per_cls, test_micro_f1, test_macro_f1 = evaluate(model, test_dl)
print(f"[TEST] mean AUROC={test_mean_auc:.4f} | microF1={test_micro_f1:.4f} | macroF1={test_macro_f1:.4f}")
print("Per-class AUROC:", {k: (None if np.isnan(v) else float(v)) for k, v in test_per_cls.items()})


Device: cuda
df shape: (112120, 12)
Y shape: (112120, 14)
Positives per class: {'Atelectasis': 11559, 'Cardiomegaly': 2776, 'Effusion': 13317, 'Infiltration': 19894, 'Mass': 5782, 'Nodule': 6331, 'Pneumonia': 1431, 'Pneumothorax': 5302, 'Consolidation': 4667, 'Edema': 2303, 'Emphysema': 2516, 'Fibrosis': 1686, 'Pleural_Thickening': 3385, 'Hernia': 227}
split sizes: 89522 11292 11306
Indexed files: 112120
Missing among first 20: 0
Splits: 89522 11292 11306


Epoch 1/10: 100%|██████████| 2798/2798 [21:34<00:00,  2.16it/s]  


[Val] mean AUROC=0.7246 | microF1=0.1520 | macroF1=0.1484 | train_loss=1.1735
✅ Saved new best to densenet121_best.pt


Epoch 2/10: 100%|██████████| 2798/2798 [18:12<00:00,  2.56it/s]


[Val] mean AUROC=0.7730 | microF1=0.1947 | macroF1=0.1758 | train_loss=1.0931
✅ Saved new best to densenet121_best.pt


...

[Val] mean AUROC=0.8386 | microF1=0.2566 | macroF1=0.2331 | train_loss=0.8449
✅ Saved new best to densenet121_best.pt


Epoch 9/10: 100%|██████████| 2798/2798 [18:20<00:00,  2.54it/s]


[Val] mean AUROC=0.8414 | microF1=0.2712 | macroF1=0.2453 | train_loss=0.8129
✅ Saved new best to densenet121_best.pt


Epoch 10/10: 100%|██████████| 2798/2798 [18:20<00:00,  2.54it/s]


[Val] mean AUROC=0.8439 | microF1=0.2697 | macroF1=0.2434 | train_loss=0.7995
✅ Saved new best to densenet121_best.pt
[TEST] mean AUROC=0.8409 | microF1=0.2799 | macroF1=0.2486
Per-class AUROC: {'Atelectasis': 0.7849813395895311, 'Cardiomegaly': 0.9434074220584963, 'Effusion': 0.8849623733879217, 'Infiltration': 0.7067924720462269, 'Mass': 0.8391730482107597, 'Nodule': 0.7907630876697277, 'Pneumonia': 0.7502949175725163, 'Pneumothorax': 0.8719545007529546, 'Consolidation': 0.8000228201090801, 'Edema': 0.8950595394746327, 'Emphysema': 0.9474297911590529, 'Fibrosis': 0.8047051601080092, 'Pleural_Thickening': 0.8111228237080977, 'Hernia': 0.9425364618342686}


In [42]:
# ========================== SETUP ==========================
# !pip install -q pandas torch torchvision scikit-learn tqdm

import os
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.metrics import roc_auc_score, f1_score
from tqdm import tqdm

torch.backends.cudnn.benchmark = True
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

# ========================== 1) PATHS ==========================
CSV_PATH = Path("./chest-xray/Data_Entry_2017.csv")    # <- change if needed
BASE     = Path("./chest-xray")                        # contains images_001, images_002, ...
assert CSV_PATH.exists(), f"CSV not found: {CSV_PATH}"
assert BASE.exists(), f"Base images folder not found: {BASE}"

# ========================== 2) LOAD CSV ==========================
df = pd.read_csv(CSV_PATH)
print("df shape:", df.shape)

# ========================== 3) LABELS + MULTI-HOT ==========================
LABELS = [
    'Atelectasis','Cardiomegaly','Effusion','Infiltration','Mass','Nodule',
    'Pneumonia','Pneumothorax','Consolidation','Edema','Emphysema',
    'Fibrosis','Pleural_Thickening','Hernia'
]

def to_multi_hot(lbl_str: str):
    y = np.zeros(len(LABELS), dtype=np.float32)
    if isinstance(lbl_str, str) and lbl_str != "No Finding":
        for t in lbl_str.split("|"):
            if t in LABELS:
                y[LABELS.index(t)] = 1.0
    return y

Y = np.stack([to_multi_hot(s) for s in df["Finding Labels"].astype(str)], axis=0)
print("Y shape:", Y.shape)
print("Positives per class:", dict(zip(LABELS, Y.sum(axis=0).astype(int))))

# ========================== 4) PATIENT-LEVEL SPLIT (80/10/10) ==========================
df["Patient ID"] = df["Patient ID"].astype(str)
bucket = df["Patient ID"].apply(lambda x: hash(x) % 10)  # 0..9
train_df = df[bucket < 8].reset_index(drop=True)
val_df   = df[bucket == 8].reset_index(drop=True)
test_df  = df[bucket == 9].reset_index(drop=True)
print("Split sizes -> Train:", len(train_df), "Val:", len(val_df), "Test:", len(test_df))

# ========================== 5) INDEX FILES ACROSS SHARDS ==========================
# Your layout: BASE / images_XXX / images / *.png
name_to_path = {}
for p in BASE.glob("images_*/images/*.png"):
    name_to_path[p.name] = str(p)

print("Indexed files:", len(name_to_path))
first20 = df["Image Index"].head(20).tolist()
missing20 = [n for n in first20 if n not in name_to_path]
print("Missing among first 20:", len(missing20))
if missing20:
    print("Example missing:", missing20[:5])

# ========================== 6) DATASET / DATALOADERS ==========================
IMG_SIZE = 384
train_tfms = transforms.Compose([
    transforms.Resize(int(IMG_SIZE*1.1)),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
val_tfms = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

def row_to_multi_hot_tensor(row):
    return torch.tensor(to_multi_hot(row["Finding Labels"]), dtype=torch.float32)

class ChestXray(Dataset):
    def __init__(self, df, index_map, tfm):
        self.df = df.reset_index(drop=True)
        self.idx = index_map
        self.tfm = tfm
    def __len__(self):
        return len(self.df)
    def __getitem__(self, i):
        r = self.df.iloc[i]
        fname = r["Image Index"]
        img_path = self.idx.get(fname)
        if img_path is None:
            raise FileNotFoundError(f"Image not indexed: {fname}")
        img = Image.open(img_path).convert("RGB")
        x = self.tfm(img)
        y = row_to_multi_hot_tensor(r)
        return x, y

train_ds = ChestXray(train_df, name_to_path, train_tfms)
val_ds   = ChestXray(val_df,   name_to_path, val_tfms)
test_ds  = ChestXray(test_df,  name_to_path, val_tfms)

BATCH_SIZE  = 32
NUM_WORKERS = 2  # set 0 if multiprocessing issues

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS, pin_memory=True)
val_dl   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
test_dl  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

print("DL sizes ->", len(train_ds), len(val_ds), len(test_ds))

# ========================== 7) MODEL ==========================
model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
in_features = model.classifier.in_features
model.classifier = nn.Linear(in_features, len(LABELS))
model = model.to(DEVICE)

# ========================== 8) LOSS (pos_weight from TRAIN) ==========================
train_multi = np.vstack(train_df["Finding Labels"].astype(str).map(
    lambda s: np.array(to_multi_hot(s), dtype=np.float32)
).values)
pos = train_multi.sum(axis=0)    # per-class positives in TRAIN
N = len(train_df)
pos = np.clip(pos, 1.0, None)    # avoid div-by-zero
pos_weight = torch.tensor((N - pos) / pos, dtype=torch.float32, device=DEVICE)

criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE == "cuda"))

# ========================== 9) EVALUATION ==========================
def evaluate(model, loader, threshold=0.5):
    model.eval()
    ys, ps = [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            p = torch.sigmoid(model(xb))
            ys.append(yb.cpu()); ps.append(p.cpu())
    ys = torch.cat(ys, 0).numpy()
    ps = torch.cat(ps, 0).numpy()

    # AUROC per class
    aurocs = []
    for c in range(len(LABELS)):
        y_c, p_c = ys[:, c], ps[:, c]
        try:
            aurocs.append(roc_auc_score(y_c, p_c))
        except ValueError:
            aurocs.append(np.nan)
    mean_auc = float(np.nanmean(aurocs))

    # F1 at fixed threshold (reference only)
    preds = (ps >= threshold).astype("int32")
    micro_f1 = f1_score(ys, preds, average="micro", zero_division=0)
    macro_f1 = f1_score(ys, preds, average="macro", zero_division=0)
    return mean_auc, dict(zip(LABELS, aurocs)), micro_f1, macro_f1

# ========================== 10) TRAIN with CHECKPOINTS ==========================
best_path = "densenet121_best.pt"
last_path = "densenet121_last.pt"
RESUME    = True
EPOCHS    = 10

def make_ckpt(epoch, best_auc):
    return {
        "epoch": epoch,
        "best_auc": best_auc,
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "scheduler_state": scheduler.state_dict(),
        "scaler_state": scaler.state_dict(),
    }

def save_last(epoch, best_auc):
    torch.save(make_ckpt(epoch, best_auc), last_path)
    print(f" Saved last: {last_path} (epoch={epoch})")

def save_best(epoch, best_auc):
    torch.save(make_ckpt(epoch, best_auc), best_path)
    print(f" Saved BEST: {best_path} (epoch={epoch}, best_auc={best_auc:.4f})")

# Resume
start_epoch = 1
best_auc = -1.0
if RESUME and os.path.exists(last_path):
    ckpt = torch.load(last_path, map_location=DEVICE)
    model.load_state_dict(ckpt["model_state"])
    optimizer.load_state_dict(ckpt["optimizer_state"])
    scheduler.load_state_dict(ckpt["scheduler_state"])
    scaler.load_state_dict(ckpt["scaler_state"])
    start_epoch = ckpt["epoch"] + 1
    best_auc    = ckpt.get("best_auc", best_auc)
    print(f"🔁 Resuming from epoch {start_epoch} (best_auc={best_auc:.4f})")
else:
    print(" Starting fresh training")

# Loop
for ep in range(start_epoch, EPOCHS + 1):
    model.train()
    running_loss = 0.0
    for xb, yb in tqdm(train_dl, desc=f"Epoch {ep}/{EPOCHS}"):
        xb, yb = xb.to(DEVICE, non_blocking=True), yb.to(DEVICE, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=(DEVICE == "cuda")):
            logits = model(xb)
            loss = criterion(logits, yb)
        scaler.scale(loss).backward()
        scaler.step(optimizer); scaler.update()
        running_loss += loss.item() * xb.size(0)

    scheduler.step()
    train_loss = running_loss / len(train_ds)

    val_mean_auc, _, val_micro_f1, val_macro_f1 = evaluate(model, val_dl)
    print(f"[Val] mean AUROC={val_mean_auc:.4f} | microF1={val_micro_f1:.4f} | macroF1={val_macro_f1:.4f} | train_loss={train_loss:.4f}")

    # Always save "last"
    save_last(ep, best_auc)

    # Save "best" if improved
    if val_mean_auc > best_auc:
        best_auc = val_mean_auc
        save_best(ep, best_auc)

# ========================== 11) TEST ==========================
if Path(best_path).exists():
    ckpt = torch.load(best_path, map_location=DEVICE)
    model.load_state_dict(ckpt["model_state"])
    print(f"✅ Loaded BEST from epoch {ckpt['epoch']} (best_auc={ckpt['best_auc']:.4f})")
else:
    print("⚠️ BEST checkpoint not found, using last model weights in memory.")

test_mean_auc, test_per_cls, test_micro_f1, test_macro_f1 = evaluate(model, test_dl)
print(f"[TEST] mean AUROC={test_mean_auc:.4f} | microF1={test_micro_f1:.4f} | macroF1={test_macro_f1:.4f}")
print("Per-class AUROC:", {k: (None if np.isnan(v) else float(v)) for k, v in test_per_cls.items()})

# ========================== 12) (Optional) SINGLE-IMAGE PREDICT ==========================
@torch.no_grad()
def predict_image(img_path, threshold=0.5, top_k=5):
    model.eval()
    img = Image.open(img_path).convert("RGB")
    x = val_tfms(img).unsqueeze(0).to(DEVICE)
    probs = torch.sigmoid(model(x)).squeeze(0).cpu().numpy()
    pred_labels = [LABELS[i] for i, p in enumerate(probs) if p >= threshold]
    top_idx = np.argsort(-probs)[:top_k]
    top = [(LABELS[i], float(probs[i])) for i in top_idx]
    return dict(zip(LABELS, map(float, probs))), pred_labels, top

# Example:
# img_example = "/chest-xray/images_001/images/00001335_006.png"
# probs, preds, top5 = predict_image(img_example, threshold=0.5, top_k=5)
# print("Predicted (>=0.5):", preds)
# print("Top-5:", top5)


Device: cuda
df shape: (112120, 12)
Y shape: (112120, 14)
Positives per class: {'Atelectasis': 11559, 'Cardiomegaly': 2776, 'Effusion': 13317, 'Infiltration': 19894, 'Mass': 5782, 'Nodule': 6331, 'Pneumonia': 1431, 'Pneumothorax': 5302, 'Consolidation': 4667, 'Edema': 2303, 'Emphysema': 2516, 'Fibrosis': 1686, 'Pleural_Thickening': 3385, 'Hernia': 227}
Split sizes -> Train: 89522 Val: 11292 Test: 11306
Indexed files: 112120
Missing among first 20: 0
DL sizes -> 89522 11292 11306
 Starting fresh training


Epoch 1/10: 100%|██████████| 2798/2798 [18:33<00:00,  2.51it/s]


[Val] mean AUROC=0.7329 | microF1=0.1702 | macroF1=0.1597 | train_loss=1.2056
 Saved last: densenet121_last.pt (epoch=1)
 Saved BEST: densenet121_best.pt (epoch=1, best_auc=0.7329)


Epoch 2/10: 100%|██████████| 2798/2798 [18:20<00:00,  2.54it/s]


[Val] mean AUROC=0.7633 | microF1=0.2145 | macroF1=0.1831 | train_loss=1.1214
 Saved last: densenet121_last.pt (epoch=2)
 Saved BEST: densenet121_best.pt (epoch=2, best_auc=0.7633)


...

[Val] mean AUROC=0.8334 | microF1=0.2470 | macroF1=0.2237 | train_loss=0.8658
 Saved last: densenet121_last.pt (epoch=8)
 Saved BEST: densenet121_best.pt (epoch=8, best_auc=0.8334)


Epoch 9/10: 100%|██████████| 2798/2798 [18:25<00:00,  2.53it/s]


[Val] mean AUROC=0.8385 | microF1=0.2567 | macroF1=0.2317 | train_loss=0.8409
 Saved last: densenet121_last.pt (epoch=9)
 Saved BEST: densenet121_best.pt (epoch=9, best_auc=0.8385)


Epoch 10/10: 100%|██████████| 2798/2798 [18:18<00:00,  2.55it/s]


[Val] mean AUROC=0.8389 | microF1=0.2577 | macroF1=0.2335 | train_loss=0.8290
 Saved last: densenet121_last.pt (epoch=10)
 Saved BEST: densenet121_best.pt (epoch=10, best_auc=0.8389)
✅ Loaded BEST from epoch 10 (best_auc=0.8389)
[TEST] mean AUROC=0.8400 | microF1=0.2718 | macroF1=0.2427
Per-class AUROC: {'Atelectasis': 0.7831616003470198, 'Cardiomegaly': 0.941590394013565, 'Effusion': 0.8831769429188057, 'Infiltration': 0.7043728759956689, 'Mass': 0.838729952319029, 'Nodule': 0.7635560244637238, 'Pneumonia': 0.7530193523358116, 'Pneumothorax': 0.8707031660967021, 'Consolidation': 0.8014594410594617, 'Edema': 0.896692123277375, 'Emphysema': 0.9405880088210847, 'Fibrosis': 0.8056480628831213, 'Pleural_Thickening': 0.8123890394150828, 'Hernia': 0.965578152600397}
